# Project Template: Regression Experiments using sentence embeddings

ISE 201, section 33

28 Mar 2024  JM Agosta

_A template to follow for your final class project._

Your presentation from this notebook should be divided into sections. Each section starts with a markdown cell that mentions the purpose of the following cells. Pls divide your computation into manageably sized cells. You can check intermediate results by an expression in the last line of the cell -- no need for a print function to enclose them.

The goal of this project is to consolidate the work you've done over the past projects, to run a regression experiment using sentence embedding data. An experiment works by

- picking an evaluation measure (a "statistic"),
- running a baseline evaluation of the regression,
- coming up with a presumed improvement to the regression that could be a generation or selection of features going into the regression,
- making a comparative evaluation to see if the improvement is "statistically" significant.

You can refer to the previous class notebooks that can be found on Google Drive, and feel free to share ideas that others have contributed in the class, that can be found in the notebooks they've shared.  

Design your notebook, with sections and visualizations that are self-explanatory (label your graph axes!) so that you make the best use of the 8 minutes you'll have to present.

## Setup. Import libraries.  Check if any need to be loaded that are missing.

In [1]:
import pandas as pd
import numpy as np
import re, os
# For vector norms, and determinants.
import numpy.linalg as la
# Plotting
import matplotlib as plt
import seaborn as sns

# javascript plots
from bokeh.plotting import figure, show
from bokeh.models import BoxAnnotation, ColumnDataSource, VBar, Span, Legend
from bokeh.io import output_notebook
from bokeh.palettes import Category10
output_notebook()

import sklearn.linear_model as sl

# Distributions
from scipy.stats import norm

# Principal component analysis
from sklearn.decomposition import PCA, TruncatedSVD
# The random number generator can be used to select a random set of columns for visualization
from numpy.random import Generator, PCG64
rng = Generator(PCG64())

Loading BokehJS ...

In [2]:
# use statsmodels to compute OLS regression standard errors
# since scikit learn doesn't compute them.
# This update fails
# os.system('python -m pip install git+https://github.com/statsmodels/statsmodels')
import statsmodels.api as sm
sm.__version__

'0.14.0'

### Set globals

In [6]:
# Global variables
EMBEDDING_DIMENSIONS = 384     # This model creates vectors of this length.
TEST_SET_SIZE = 500
BOOTSTRAP_RESAMPLES = 200
RELOAD_PACKAGE = False         # Reload the embedding package
USE_COLAB = False              # Load the data from your Google Drive

In [5]:
# If you need to rerun the sentence embeddings transformer, you'll have to reload the package since
# its not included in Colab.  Set RELOAD_PACKAGE to True to reload it.


# We'll use this dataset of 3000 labeled text samples.
# https://archive.ics.uci.edu/dataset/331/sentiment+labelled+sentences

# For converting the text into vectors, use this llm package
# See https://pypi.org/project/sentence-transformers/
try:
  from sentence_transformers import SentenceTransformer
except:
  if RELOAD_PACKAGE:
    os.system('python -m pip install sentence-transformers')
    from sentence_transformers import SentenceTransformer
  else:
      print('import SentenceTransformer failed')


## Load the data, and run the sentence tranformers on it, if needed

We've been using the data set as indexed by embeddings from ---a set of 3,000 items from 3 different sources, - "IMDB", Amazon, and Yelp reviews, on which the 'all-MiniLM-L6-v2' sentenceTransformer model generates 384 features.

If you need to download this dataset, go to:

    https://archive.ics.uci.edu/dataset/331/sentiment+labelled+sentences

If you want to use a larger set, here's an NLP data set from Kaggle  of 50,000 items. One can find them on the Kaggle site:

To download the dataset, go to:

https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

You would need to download the "IMDB Dataset.csv" file - 66.21MB


In [7]:
# Run this if you want to run on Colab and use the already featurized data

if USE_COLAB and not os.path.exists('/content/drive'):
  from google.colab import drive
  drive.mount('/content/drive/')
  os.chdir('/content/drive/MyDrive/ISE201/notebooks/data')
else:
  os.chdir('./sentiment_labelled_sentences')
os.listdir(os.getcwd())

['.DS_Store',
 'amazon_cells_labelled.txt',
 'imdb_labelled.txt',
 'readme.txt',
 'sentence_embeddings.parquet',
 'yelp_labelled.txt']

In [9]:
# Load the embedding vectors dataset
if os.environ['HOME'] == '/home/jma':
    featurized_parquet_file = 'sentence_embeddings.parquet'
elif USE_COLAB:
  # Find the file on Googe Drive
    featurized_parquet_file = './3000_sentence_embeddings.parquet'

text_df = pd.read_parquet(featurized_parquet_file)

# Extract the outcome y vector
outcomes = text_df['sentiment'].values

# Extract the vector field and expand it to multiple rows.
n_samples, embedding_dim = text_df.shape
data_ar = np.empty((n_samples, EMBEDDING_DIMENSIONS), 'float')
for i in range(n_samples):
    x = text_df.values[i,2]
    data_ar[i] = x

# Check the data shapes: raw NLP data, and itsEMB sentence embeddings, outocomes
text_df.shape, data_ar.shape, outcomes.shape

((2748, 3), (2748, 384), (2748,))

In [44]:
# Split the data into a learning and holdout set
# Create a random sample, with replacement
sample_size = outcomes.shape[0]- TEST_SET_SIZE
learning_set = np.random.choice(len(data_ar),\
                                sample_size,\
                                replace=False)
learning_data = data_ar[learning_set,:]
learning_outcome = text_df['sentiment'].values[learning_set]

# Run predictions against a hold-out set
not_learning_set = list(set(text_df.index).difference(set(learning_set)))  # TODO check if this set is correct
holdout_data = data_ar[not_learning_set,:]
holdout_data = sm.add_constant(holdout_data)
# holdout_data = sm.add_constant(holdout_data)
holdout_actuals = text_df['sentiment'].values[not_learning_set]
learning_data.shape, len(learning_set), sample_size, learning_outcome.shape

((2248, 384), 2248, 2248, (2248,))

## Pick an evaluation metric & create visualization graphics

In [92]:
# An in-sample metric
def AIC(the_fit):
  'Use this to compute AIC on different models.'
  return 2* (the_fit.df_model+1) - 2*the_fit.llf

# A metric to compare out-of-sample error
def one_zero_error(predicted, actual, test_threshold = 0.5):
  'The predicted is a continuous value, so it needs to be thresholded. '
  test_df = pd.DataFrame(dict(prediction=pd.Series(predicted >= test_threshold), actual=actual))
  test_error = sum(test_df.prediction != actual)/len(actual)
  return test_error

In [12]:
# A plot that shows the difference between the normal approximation interval and an equivalent coverage empirical interval,

def expand_range(low, high, by=2.0):
    mid = (low + high)/2
    half_width = by * abs(high - low)/2
    return (mid - half_width, mid + half_width)

# Compute two different interval estimates
# The first assuming a normal distribution, using standard deviation
# The second computing the equivalent quantiles 0.025 and 0.975, from the empirical data. 
# Note: For extreme distributions, such as those on a closed or half interval, this may not be the best approximation
def interval_estimate(data_list):
    data_ar = np.array(data_list)
    mn = data_ar.mean()
    sd = data_ar.std()
    return {'mean': mn, 'sd': sd,
        'normal_interval': (mn-1.96*sd, mn+1.96*sd),
            'quantile_interval': (np.percentile(data_ar, 2.5), np.percentile(data_ar, 97.5))}

def plot_intervals(the_sample, bin_ct = 50, subtitle = ''):
    normal_approx = interval_estimate(the_sample)
    lower_limit = min(normal_approx['normal_interval'][0], normal_approx['quantile_interval'][0])
    upper_limit = max(normal_approx['normal_interval'][1], normal_approx['quantile_interval'][1])
    lower_plot_limit, upper_plot_limit = expand_range(lower_limit, upper_limit)
    print('Limits ', lower_plot_limit, upper_plot_limit)
    p = figure(width = 800, height = 300, x_range = (lower_plot_limit,  upper_plot_limit ), title='Normal versus Empirical Bootstrap Interval Estimates.\n' + subtitle)

    hist, bin_edges = np.histogram(the_sample, bins=bin_ct, density=True)
    hist_df = pd.DataFrame(dict(density=hist, rv= bin_edges[:-1]), columns=['rv', 'density'])
    # Shade the background corresponding to the normal interval
    box = BoxAnnotation(left=normal_approx['normal_interval'][0], right=normal_approx['normal_interval'][1], bottom=0, top=max(1, max(hist)), fill_alpha=0.2, fill_color='#D55E00')
    p.add_layout(box)
    bar_width = (bin_edges[-1] - bin_edges[0])/bin_ct
    hist_src = ColumnDataSource(hist_df)

    glyph = VBar(x='rv', top='density', bottom=0, width=bar_width, fill_color='limegreen')
    p.add_glyph(hist_src, glyph)
    # Overlay a selection of the histogram within the estimate interval
    interval_src = ColumnDataSource(hist_df[(hist_df.rv > normal_approx['quantile_interval'][0]) & (hist_df.rv < normal_approx['quantile_interval'][1])])
    glyph = VBar(x='rv', top='density', bottom=0, width=bar_width, fill_color='darkgreen')
    p.add_glyph(interval_src, glyph)
    # Overlay the normal distribution within the interval.
    x = np.linspace(normal_approx['normal_interval'][0], normal_approx['normal_interval'][1])
    p.line(x=x, y=norm.pdf(x, loc= normal_approx['mean'], scale = normal_approx['sd']))
    return p

## Run the baseline regression

In [13]:
# Run an OLS regression on the full data.
def regress(y, X):
  design_matrix = sm.add_constant(X)
  ols_model = sm.OLS(y, design_matrix)
  #print(ols_model.fit().summary())
  return ols_model.fit()

def print_eval_summary(the_fit):
  # Just show the evaluation table, without coefs (tables[1] give coefs)
  s = the_fit.summary()
  print (s.tables[0])
  return None

In [14]:
fit = regress(outcomes, data_ar)
print_eval_summary(fit)
'AIC: ', AIC(fit)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.709
Model:                            OLS   Adj. R-squared:                  0.662
Method:                 Least Squares   F-statistic:                     15.06
Date:                Sun, 21 Apr 2024   Prob (F-statistic):               0.00
Time:                        16:27:30   Log-Likelihood:                -299.62
No. Observations:                2748   AIC:                             1365.
Df Residuals:                    2365   BIC:                             3632.
Df Model:                         382                                         
Covariance Type:            nonrobust                                         


('AIC: ', 1365.2353739571927)

In [15]:
# Compute in-sample error
one_zero_error(fit.predict(), outcomes, test_threshold = 0.5)

0.06186317321688501

In [28]:
# Look at some of the beta coefficients
pd.DataFrame(fit.summary().tables[1])

,0,1,2,3,4,5,6
0,,coef,std err,t,P>|t|,[0.025,0.975]
1,const,0.5822,0.056,10.336,0.000,0.472,0.693
2,x1,-0.2610,0.235,-1.112,0.266,-0.721,0.199
3,x2,0.4099,0.237,1.727,0.084,-0.056,0.875
4,x3,-0.2591,0.268,-0.967,0.333,-0.784,0.266
...,...,...,...,...,...,...,...
381,x380,-0.0595,0.275,-0.216,0.829,-0.599,0.480
382,x381,0.5904,0.229,2.576,0.010,0.141,1.040
383,x382,0.8383,0.230,3.650,0.000,0.388,1.289
384,x383,0.1760,0.231,0.760,0.447,-0.278,0.630


## An example Interval estimate for out-of-sample 0-1 error via Bootstrap re-sampling

This demonstrates the 0-1 error rate interval estimate, by resampling the training predictor, then measuring accuracy on the holdout set. This may be overkill, as a way to estimate the variation in the holdout error. 

Note that the non-parametric interval does slightly worse than its normal approximation interval. 

In [16]:
# Create a large number of bootstrap replications
# Each column in "resamplings" is a set of indexes into the rows of the training set, resampled with replacement.

def bootstrap_replicates(the_sample, B=BOOTSTRAP_RESAMPLES):
    replicates = np.zeros((len(the_sample), B), dtype='int')
    for k in range(B):
        replicates[:,k] = np.random.choice(the_sample, len(the_sample), replace=True).astype('int')
    return replicates

resamplings = bootstrap_replicates(list(range(sample_size)))
resamplings.shape

(2248, 200)

In [34]:
# Bootstrap the zero-one error on a hold-out set.

# Create a large number of bootstrap replications
resampled_errors = np.zeros(BOOTSTRAP_RESAMPLES)

# Retrain the model on the resampled learning set,
# and compute the hold-out set error for each resample
print('learning AIC: ')
for k in range(BOOTSTRAP_RESAMPLES):
  resampled_data  = learning_data[resamplings[:,k],:]
  retraining_fit = regress(learning_outcome[resamplings[:,k]], resampled_data) # [resamplings[:,k]], resampled_data)
  print(f'{AIC(retraining_fit):.5}', end=', ')
  if k % 10 == 0:
    print()


  # Predict on the hold-out set
  hold_out_predict = retraining_fit.predict(holdout_data)
  resampled_errors[k] = one_zero_error(hold_out_predict, holdout_actuals)


learning AIC: 
794.23, 
604.38, 816.83, 829.81, 777.38, 780.35, 859.81, 748.26, 704.3, 765.55, 637.26, 
623.55, 791.59, 773.78, 684.01, 774.25, 708.74, 745.52, 573.65, 672.25, 845.33, 
637.89, 618.32, 828.41, 709.26, 843.14, 767.33, 711.81, 673.35, 732.88, 745.62, 
663.39, 624.02, 663.51, 825.88, 760.15, 684.73, 827.12, 671.07, 878.4, 751.46, 
677.22, 731.73, 711.14, 962.41, 923.41, 868.85, 761.49, 861.27, 715.8, 778.42, 
701.67, 745.97, 708.61, 763.56, 691.84, 892.71, 771.99, 746.75, 789.35, 707.19, 
780.32, 716.05, 706.8, 731.04, 791.6, 760.45, 519.82, 754.02, 679.01, 624.79, 
620.24, 902.89, 805.15, 767.02, 666.16, 813.3, 723.5, 806.52, 782.63, 896.91, 
694.02, 750.04, 882.43, 733.25, 691.07, 785.92, 696.73, 793.8, 735.62, 829.84, 
749.75, 735.34, 663.74, 774.72, 792.22, 707.64, 798.3, 675.91, 699.88, 602.09, 
598.65, 647.88, 826.79, 656.08, 825.62, 790.34, 687.93, 844.19, 812.99, 770.78, 
790.27, 813.28, 946.93, 835.24, 773.26, 570.89, 937.23, 699.55, 703.61, 580.17, 
680.58, 801.4

In [36]:
# What to the bootstrap errors look like?
pd.DataFrame(resampled_errors).describe()

,0
count,200.000000
mean,0.129892
std,0.018329
min,0.080645
25%,0.118280
50%,0.129032
75%,0.145161
max,0.182796


In [37]:
#Plot the error distribution, and compute the interval estimate limit.
p = plot_intervals(resampled_errors)
show(p)

Limits  0.05775321412926951 0.20297115084579906


##  Generate selected features, for an alternative model

A crude way to select features for linear regression is to sort the variables by the largest correlation with the dependent variable.  Let's see how the holdout error changes as variables are added to the linear regression. 

In [109]:
# Use a dot product to approximate the covariance

feature_cov = learning_data.T @ learning_outcome / la.norm(learning_data, axis=0)
pd.DataFrame(feature_cov).describe()

,0
count,384.000000
mean,0.067043
std,8.472924
min,-22.665340
25%,-6.630662
50%,0.079711
75%,6.438096
max,18.791371


In [116]:
# Ordering of the variables by absolute value of cov
feature_ordering =   np.argsort(feature_cov) # np.argsort(np.abs(feature_cov)) #
ordered_features = feature_cov[feature_ordering[::-1]]        # [::-1] means reverse the ordering
pd.DataFrame(ordered_features[:10])

,0
0,18.791371
1,17.707732
2,17.387248
3,17.130931
4,17.064447
5,16.678416
6,16.626368
7,16.203136
8,15.922810
9,15.832757


In [117]:
# Compute AIC as a function of the number of variables, using no selection ordering
unsorted_AIC = np.zeros(380)
for var_count in range(8,380):
    fit = regress(learning_outcome, learning_data[:,:var_count]) 
    unsorted_AIC[var_count] = AIC(fit)

In [123]:
# Compute AIC as a function of the number of variables
sorted_AIC = np.zeros([380, 2])
# Hmm - using the negative valued cov does slightly better. 
sorted_learning_data = learning_data[:,feature_ordering] # learning_data[:,feature_ordering[::-1]]
for var_count in range(8, 380):
    fit = regress(learning_outcome, sorted_learning_data[:,:var_count]) 
    sorted_AIC[var_count, 0] = var_count
    sorted_AIC[var_count, 1] = AIC(fit)

In [124]:
f = figure(width = 700, height = 300,  x_range = (8,  EMBEDDING_DIMENSIONS ), title='AIC for cov-selected variable sets') # ,
f.line(x=list(range(380)), y=unsorted_AIC, color = 'darkred')
f.line(x=sorted_AIC[:,0], y=sorted_AIC[:,1], color = 'red')
hline = Span(location=1365, dimension='width', line_width= 0.6) # , fill_alpha=0.4)
f.renderers.extend([hline])
show(f)

## Compare baseline and alternative models

In [228]:
#TBD